In [310]:
print("hello")

hello


In [311]:
from streamlit_chat import message
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain,ConversationChain
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile
# As openAi api is not working i am using huggingface access token, it is a bit slower though
from langchain import HuggingFaceHub


In [312]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [313]:
#load pdf files
loader = PyPDFDirectoryLoader("documents/")
data = loader.load()

In [314]:
print(data)

[Document(page_content="Sujan\nSharma\nML\nEngineer\nAssociate\nTrainee\n✉\nsujan.sharm a@ fusem achines.com\n|\nLinkedin\n|\nGitH ub\nM achine\nLearning\nEngineer\nTrainee\nw ith\na\nBachelor’s\nD egree\nin\nCom puter\nEngineering.\nA\ndedicated\nand\ndetail-oriented\nindividual\nw ho\nis\nalw ays\neager\nto\nlearn\nand\ntake\non\nnew\nchallenges.\nTransitted\nthe\nﬁeld\nfrom\na\nFront\nEnd\nD evelopm ent\nto\nthe\nﬁeld\nof\nM achine\nlearning,\nI\nbring\nover\na\nyear's\nw orth\nof\nexperience\nin\ncreating\nm any\nuser\nengaging\nw eb\ninterfaces.\nPROJECT\nEXPERIENCE\nMajor\nProject\n-\nPoint\nOut\nCrops\n●\nPractical\nIm plem entation\nand\nM odel\ncom parison\nand\nused\nm ost\nof\nthe\nconcepts\nregarding\nD eep\nlearning\nand\nApp\ndevelopm ent\nalong\nw ith\nIO T.\n●\nGot\nthe\nhighest\naccuracy\nof\n93.47%\nw ith\nResnet50\nM odel.\n●\nUsed\nRaspberry\nPi\nto\nhost\nthe\nclassiﬁcation\nm odel\nand\nautom ate\nthe\nrotation\nand\nspray\nof\nthe\nm otors.\n●\nM ade\nthe\nsim pl

In [315]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)
text_chunks

[Document(page_content="Sujan\nSharma\nML\nEngineer\nAssociate\nTrainee\n✉\nsujan.sharm a@ fusem achines.com\n|\nLinkedin\n|\nGitH ub\nM achine\nLearning\nEngineer\nTrainee\nw ith\na\nBachelor’s\nD egree\nin\nCom puter\nEngineering.\nA\ndedicated\nand\ndetail-oriented\nindividual\nw ho\nis\nalw ays\neager\nto\nlearn\nand\ntake\non\nnew\nchallenges.\nTransitted\nthe\nﬁeld\nfrom\na\nFront\nEnd\nD evelopm ent\nto\nthe\nﬁeld\nof\nM achine\nlearning,\nI\nbring\nover\na\nyear's\nw orth\nof\nexperience\nin\ncreating\nm any\nuser\nengaging\nw eb\ninterfaces.\nPROJECT\nEXPERIENCE\nMajor\nProject\n-\nPoint\nOut\nCrops\n●\nPractical\nIm plem entation\nand\nM odel\ncom parison\nand\nused\nm ost\nof\nthe\nconcepts\nregarding\nD eep\nlearning\nand\nApp\ndevelopm ent\nalong\nw ith\nIO T.\n●\nGot\nthe\nhighest\naccuracy\nof\n93.47%\nw ith\nResnet50\nM odel.\n●\nUsed\nRaspberry\nPi\nto\nhost\nthe\nclassiﬁcation\nm odel\nand\nautom ate\nthe\nrotation\nand\nspray\nof\nthe\nm otors.\n●\nM ade\nthe\nsim pl

In [316]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [317]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store

In [318]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [319]:
from langchain import HuggingFaceHub
from tokenconstant import HUGGINGGACE_API_TOKEN

In [320]:
llm = HuggingFaceHub(huggingfacehub_api_token=HUGGINGGACE_API_TOKEN,
                         repo_id=model_id,
                         model_kwargs={"temperature": 0.1, "max_new_tokens": 200})
# combine_docs_chain = StuffDocumentsChain(verbose=True)


/media/sujan/Sujan/1FuseMachinesFiles/SkillUp-LLMs/Langchain/stream-lit-app/env/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [321]:
# template = """
#     Your task is to help a Hiring manager of the Tech Company.
#     The name of the user is given at top part of the document mostly so find the name and if the name is asked then return in single word.
#     If there is Question related to education is asked then you can find the answer in Education portion.
#     If there is Question related to work experience is asked then you can find the answer in Experience portion.
#     If there is Question related to projects is asked then you can find the answer in Projects portion.
#     If there is Question related to technical Skills is asked then you can find the answer in Skills/ Technical Skills portion.
#     If there is Question related to certificates from courses or videos is asked  then you can find the answer in Certificates portion.
#     If there is Question related to interested areas or hobbies is asked then you can find the answer in Interest or Hobby portion.
#     If there is Question related to education is asked then you can find the answer in Education portion.
#     If there is Question related to education is asked then you can find the answer in Education portion.
#     The date are present in start to end format for Experience and Education portion.
    
# {context}

# Question: {question}
# """
    # You should find one answer for any question is asked in above areas, not more than that.


In [322]:
from langchain.prompts.prompt import PromptTemplate
template = """
    Your task is to help a Hiring manager of the Tech Company.
    The name of the user is given at top part of the document mostly so find the name and if the name is asked then return in single word.
    For Education related question get answer from Education portion,
    For projects related question get answer from projects portion,
    For certificates related question get answer from certificates portion,
    For interested areas or hobbies related question get answer from Interest or Hobby portion,
    For work experience related question get answer from work experience portion,
    For technical Skills related question get answer from technical Skills portion,
    The date are present in start to end format for Experience and Education portion. 
    There may not exist any of above portion then return Relevent information is not provided.
    
{context}

Question: {question}
"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [345]:
import re

query = "What is the name of the user’s first project?"

result = qa.run(query)
print("====================================")
# print(result)
print("==================================== \n")

lines = result.strip().split('\n')

match = re.search(r'Answer:(.*?)Question:', result, re.DOTALL)

first_answer = ""
rest_of_questions = []

i = 0
while i < len(lines):
    if lines[i].startswith('Answer'):
        if match:
            first_answer = match.group(1).strip()
        else:
            first_answer = lines[i][8:]
        i += 1
        while i < len(lines) and not lines[i].startswith('Question'):
            i += 1
    elif lines[i].startswith('Question'):
        rest_of_questions.append(lines[i][10:])
        i += 1
    else:
        i += 1

print(first_answer)
print("------------------------")
print(rest_of_questions)


Point Out Crops
------------------------
[]


In [324]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)

In [325]:
system_msg_template = SystemMessagePromptTemplate.from_template(template="""Answer the question as truthfully as possible using the provided context, 
and if the answer is not contained within the text below, say 'I don't know',
                                                                
                                                                  'Your task is to help a Hiring manager of the Tech Company.
    The name of the user is given at top part of the document mostly so find the name and if the name is asked then return in single word.
    Find the corresponding answer from different portion of the provided.
    If there is anything related to education is asked then you can find the answer in Education portion.
    If there is anything related to work experience is asked then you can find the answer in Experience portion.
    If there is anything related to projects is asked then you can find the answer in Projects portion.
    If there is anything related to technical Skills is asked then you can find the answer in Skills/ Technical Skills portion.
    If there is anything related to certificates from courses or videos is asked  then you can find the answer in Certificates portion.
    If there is anything related to interested areas or hobbies is asked then you can find the answer in Interest or Hobby portion.
    If there is anything related to education is asked then you can find the answer in Education portion.
    If there is anything related to education is asked then you can find the answer in Education portion.
    The date are present in start to end format for Experience and Education portion.'
                                                                """)


human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")

prompt_template = ChatPromptTemplate.from_messages([system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])

memory = ConversationBufferMemory(return_messages=True)#

conversation = ConversationChain(memory=memory, prompt=prompt_template, llm=llm, verbose=True )



In [326]:
query ="Who is sujan sharma?"
conversation.predict(input=f"Query:{query}")



> Entering new ConversationChain chain...
Prompt after formatting:
System: Answer the question as truthfully as possible using the provided context, 
and if the answer is not contained within the text below, say 'I don't know',
                                                                
                                                                  'Your task is to help a Hiring manager of the Tech Company.
    The name of the user is given at top part of the document mostly so find the name and if the name is asked then return in single word.
    Find the corresponding answer from different portion of the provided.
    If there is anything related to education is asked then you can find the answer in Education portion.
    If there is anything related to work experience is asked then you can find the answer in Experience portion.
    If there is anything related to projects is asked then you can find the answer in Projects portion.
    If there is anything related to technic

'\n\nAI: Sujan Sharma is the name of the user.'

In [327]:
query = "What is his name?"

In [328]:
qa.run(query)

'\nAnswer: Sujan Sharma'

In [329]:
from langchain.document_loaders import TextLoader


In [330]:
loader = TextLoader("documents/nepalwekipedia.txt")
txtdata = loader.load()
txtdata

[Document(page_content='', metadata={'source': 'documents/nepalwekipedia.txt'})]

In [331]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)
text_chunks

[Document(page_content="Sujan\nSharma\nML\nEngineer\nAssociate\nTrainee\n✉\nsujan.sharm a@ fusem achines.com\n|\nLinkedin\n|\nGitH ub\nM achine\nLearning\nEngineer\nTrainee\nw ith\na\nBachelor’s\nD egree\nin\nCom puter\nEngineering.\nA\ndedicated\nand\ndetail-oriented\nindividual\nw ho\nis\nalw ays\neager\nto\nlearn\nand\ntake\non\nnew\nchallenges.\nTransitted\nthe\nﬁeld\nfrom\na\nFront\nEnd\nD evelopm ent\nto\nthe\nﬁeld\nof\nM achine\nlearning,\nI\nbring\nover\na\nyear's\nw orth\nof\nexperience\nin\ncreating\nm any\nuser\nengaging\nw eb\ninterfaces.\nPROJECT\nEXPERIENCE\nMajor\nProject\n-\nPoint\nOut\nCrops\n●\nPractical\nIm plem entation\nand\nM odel\ncom parison\nand\nused\nm ost\nof\nthe\nconcepts\nregarding\nD eep\nlearning\nand\nApp\ndevelopm ent\nalong\nw ith\nIO T.\n●\nGot\nthe\nhighest\naccuracy\nof\n93.47%\nw ith\nResnet50\nM odel.\n●\nUsed\nRaspberry\nPi\nto\nhost\nthe\nclassiﬁcation\nm odel\nand\nautom ate\nthe\nrotation\nand\nspray\nof\nthe\nm otors.\n●\nM ade\nthe\nsim pl